# Packaging an Overlay

This notebook will demonstrate how to package an Overlay. This notebook depends on the previous three notebooks:

2. [Creating a Vivado HLS Core](2-Creating-A-Vivado-HLS-Core.ipynb)
3. [Building a Vivado Bitstream](3-Building-A-Bitstream.ipynb)
4. [Using an HLS core in PYNQ](4-Using-an-HLS-core-in-PYNQ.ipynb)

In this notebook you will generate the following file inside the `tutorial` directory:
   - `setup.py`                     A pip setup file for installing PYNQ packages

## Outputs from Previous Steps

The first two critical components of a PYNQ overlay are a `.tcl` script file and a bitfile. These files should have been created in **[Creating a FPGA Bitstream](2-Creating-A-Bitstream.ipynb)** and with the names `stream.tcl` and `stream.bit`. 

In addition, you will need the `__init__.py` and `stream.py` python files created in **[Using an HLS core in PYNQ](4-Using-an-HLS-core-in-PYNQ.ipynb)**. 

Verify that the following files are in the `tutorial/pynqhls/stream` folder of your PYNQ-HLS repository on your **host computer.**
       
   1. `stream.tcl`
   2. `stream.bit`
   3. `stream.py`
   4. `__init__.py`
   
Using [SAMBA](http://pynq.readthedocs.io/en/v2.0/getting_started.html#accessing-files-on-the-board), or SCP, copy these files from your host machine to the directory `/home/xilinx/PYNQ-HLS/tutorial/pynqhls/` on your PYNQ board.

Alternatively, you can skip this step by running the command below: 

In [ ]:
!cp /home/xilinx/PYNQ-HLS/pynqhls/stream/stream.tcl /home/xilinx/PYNQ-HLS/tutorial/pynqhls/stream/
!cp /home/xilinx/PYNQ-HLS/pynqhls/stream/stream.bit /home/xilinx/PYNQ-HLS/tutorial/pynqhls/stream/
!cp /home/xilinx/PYNQ-HLS/pynqhls/stream/stream.py /home/xilinx/PYNQ-HLS/tutorial/pynqhls/stream/
!cp /home/xilinx/PYNQ-HLS/pynqhls/stream/__init__.py /home/xilinx/PYNQ-HLS/tutorial/pynqhls/stream/

## Notebooks Folder

Finally, we need to create a `notebooks` folder and populate its contents.

Inside of this folder we need to create a single blank file named `PLACEHOLDER`. 

Alternatively, you can copy the notebook folder from the the stream overlay provided with the tutorial:

In [ ]:
!cp -r /home/xilinx/PYNQ-HLS/pynqhls/stream/notebooks /home/xilinx/PYNQ-HLS/tutorial/pynqhls/stream/notebooks

## Automating Installation

The next step is to automate the installation using [PIP](https://pip.pypa.io).

In the previous notebook you added your `streamOverlay` class to the Python Search path. By creating a `setup.py` file you will be able to run your overlay from any Python shell without modifying the search path.

To do this, we will create a `setup.py` script in the repository.

You cannot skip this step, but fortunately, we provide working code for you.

### Creating a setup.py file

A `setup.py` file is the central part of distributing a python package using PIP. In this step we will create a simple `setup.py` script that executes the following steps:

1. Installs the notebooks folder as a folder with the name `HLS-Tutorial-Output` in the Jupyter Home directory (`/home/xilinx/jupyter_notebooks` on a PYNQ board)
2. Installs the overlay (`stream.bit` file, `stream.tcl` file, `__init__.py`, and `stream.py`) from the previous step into the python `site-packages` folder.

More documentation about setup.py and setup scripts can be found on the [distutils documentation page](https://docs.python.org/3/distutils/setupscript.html).

Copy and paste the following cell into `setup.py` inside `/home/xilinx/PYNQ-HLS/tutorial/`:

In [ ]:
from setuptools import setup
import os

jupyter_dest = '/home/xilinx/jupyter_notebooks'

# Find all of the stream notebooks in the stream_src path                                                                                                                                                                                     
stream_src = 'pynqhls/stream/notebooks/'
stream_dest = os.path.join(jupyter_dest, 'HLS-Stream-Output')
stream = [os.path.join(stream_src, f)
             for f in os.listdir(stream_src)]

# Find all of the tutorial notebook pictures in the pictures_src path                                                                                                                                                                         
stream_pictures_src = os.path.join(stream_src, 'pictures')
stream_pictures_dest = os.path.join(stream_dest, 'pictures')
stream_pictures = [os.path.join(stream_pictures_src, f) for f in os.listdir(stream_pictures_src)]

tutorials.remove(pictures_src)
stream.remove(stream_pictures_src)

setup(name='pynq-hls',
      version='0.1',
      description="A simple package describing how to create a PYNQ\                                                                                                                                                                          
            bitstream with HLS cores",
      author='Dustin Richmond',
      author_email='drichmond@eng.ucsd.edu',
      url='https://github.com/drichmond/PYNQ-HLS/',
      license='BSD-3',
      data_files = [(tutorials_dest, tutorials),
                    (pictures_dest, pictures),
                    (stream_dest, stream),
                    (stream_pictures_dest, stream_pictures)],
      package_dir = {'pynqhls_tutorial': './pynqhls'},
      packages=['pynqhls_tutorial', 'pynqhls_tutorial.stream', 'pynqhls_tutorial.memmap'],
      package_data={'':['*.bit', '*.tcl']},
      install_requires=['pynq'],
      dependency_links=['http://github.com/xilinx/PYNQ.git@v2.0#egg=pynq'],
)

The `setup.py` file starts with a set of variable declarations:

- The variable `jupyter_dest` defines the path of the Jupyter Notebooks directory on the PYNQ board. 
- The variable `stream_src` defines the location of the `.ipynb` notebook files for delivery, and `stream_dest` defines the destination. Likewise for `stream_pictures_src` and `stream_pictures_dest`.

Inside the call to setup() we pass the arguments [data_files](https://docs.python.org/3.6/distutils/setupscript.html#installing-additional-files), [packages and package_dir](https://docs.python.org/3.6/distutils/setupscript.html#listing-whole-packages), and [package_data](https://docs.python.org/3.6/distutils/setupscript.html#installing-package-data). 

- `data_files` is a list of Relative-Source and Destination-Path tuples used to install extra files
- `packages` is a list of Python packages to install
- `package_dir` is a dictonary mapping package names to directory locations
- `package_data` is a dictionary containing a package and list of relative files that should be included in that package

In order to install this as a PYNQ package using `setup.py` we need to modify all four variables as shown above

Once you have saved `setup.py` in `/home/xilinx/PYNQ-HLS/tutorial` you will be able to install the Stream Overlay you have created on your PYNQ board using the cell below:

In [ ]:
!pip3.6 install /home/xilinx/PYNQ-HLS/tutorial

When the command has completed reload this notebook and run the following command: 

In [3]:
from pynqhls_tutorial import streamOverlay

overlay = streamOverlay('stream.bit')

If you are successful, you can re-run the same tests from the previous notebook:

In [ ]:
signal = range(0, 1000)
coeffs = [1, 0, 0, 0, 0, 0, 0, 0, 0]
output = overlay.run(coeffs, signal)

test = [s == o for (s, o) in zip(signal, output)]

if False in test:
    print("Test Failed!")
else:
    print("Test Passed")

## Installing from Git

The steps we have shown above install a Python package from a local directory. To install a Python Package from Git you must provide a `setup.py` file in the root directory. 

These tutorials demonstrate one example of using a `setup.py` file to distribute a PYNQ package. The `setup.py` file we use is shown below: 

In [ ]:
from setuptools import setup

import os

jupyter_dest = '/home/xilinx/jupyter_notebooks'

# Find all of the tutorial notebooks in the tutorials_src path                                                                                                                                                                                
tutorials_src = 'tutorial/notebooks/'
tutorials_dest = os.path.join(jupyter_dest, 'HLS-Tutorial')
tutorials = [os.path.join(tutorials_src, f)
             for f in os.listdir(tutorials_src)]

# Find all of the tutorial notebook pictures in the pictures_src path                                                                                                                                                                         
pictures_src = os.path.join(tutorials_src, 'pictures')
pictures_dest = os.path.join(tutorials_dest, 'pictures')
pictures = [os.path.join(pictures_src, f) for f in os.listdir(pictures_src)]

# Find all of the stream notebooks in the stream_src path                                                                                                                                                                                     
stream_src = 'pynqhls/stream/notebooks/'
stream_dest = os.path.join(jupyter_dest, 'HLS-Stream')
stream = [os.path.join(stream_src, f)
             for f in os.listdir(stream_src)]

# Find all of the tutorial notebook pictures in the pictures_src path                                                                                                                                                                         
stream_pictures_src = os.path.join(stream_src, 'pictures')
stream_pictures_dest = os.path.join(stream_dest, 'pictures')
stream_pictures = [os.path.join(stream_pictures_src, f) for f in os.listdir(stream_pictures_src)]

tutorials.remove(pictures_src)
stream.remove(stream_pictures_src)

setup(name='pynq-hls',
      version='0.1',
      description="A simple package describing how to create a PYNQ\                                                                                                                                                                          
            bitstream with HLS cores",
      author='Dustin Richmond',
      author_email='drichmond@eng.ucsd.edu',
      url='https://github.com/drichmond/PYNQ-HLS/',
      license='BSD-3',
      data_files = [(tutorials_dest, tutorials),
                    (pictures_dest, pictures),
                    (stream_dest, stream),
                    (stream_pictures_dest, stream_pictures)],
      packages=['pynqhls', 'pynqhls.stream', 'pynqhls.memmap'],
      package_data={'':['*.bit', '*.tcl']},
      install_requires=['pynq'],
      dependency_links=['http://github.com/xilinx/PYNQ.git@v2.0#egg=pynq'],
)
